### How to use the Data Lab *Auth Client* Service

*Revised: Dec 22, 2017*

This notebook documents how to use the Data Lab authorization system via the *Auth Client* service. This can be done either from a Python script or from the command line using the <i>datalab</i> command.

While most Data Lab services may be accessed anonymously, others require a valid user account in order to take advantage of resources such as <i>Virtual Storage</i> or <i>Compute Services</i>.  User accounts may be created on the main Data Lab website and are not discussed here.

Assuming an account has been created, the <i><b>authClient</b></i> interface can be used to authenticate yourself to the Data Lab to obtain an identity token used to access the resources allocated to the account.  This is typically the first step in any notebook or application, the returned token is then passed to every other interface method that requires authenticated access to a resource or service.

A complete summary of the *AuthClient* interface is given below.

***

#### Example User Login 

Most notebooks will begin by importing whatever packages are required.  In this case we'll want to import the *authClient* interface from the *dl* (i.e. Data Lab) package.  The next step is to call the *login()* method with your account name and password to obtain an identity token:

In [1]:
# Standard notebook imports
from __future__ import print_function
import getpass
from dl import authClient, queryClient, storeClient

In [2]:
# Login using your user name and password, validate the returned
# token or else print the returned error message.
token = authClient.login ('demo00',getpass.getpass('Account Password: '))
if not authClient.isValidToken (token):
    print ('Error: invalid user login (%s)' % token)
else:
    print ("Login token:   %s" % token)

Account Password: ········
Login token:   demo00.1018.1018.$1$Dyy.y/qT$z7u.OUM3y69TOXOV.zRL4.


The `login()` method will return either a valid token or an error message.  By calling the `isValidToken()` method it is possible to trap any login errors in the task.  The use of the '*getpass*' function means passwords do not need to be exposed and notebooks can be shared more freely.

<font color='red'><h4>Security Note</h4></font>

<font color='green'>
Once you have logged in successfully, an identity token is saved on the server running the application (the Jupyter notebook server in this case) and will be offered for subsequent logins when no password is provided.  This means that notebooks may be shared to use a valid account without sharing the password in the notebook.  You can use the `logout()` method (see below) to remove this token from the server manually, but as an added precaution, tokens will not be used on public notebook servers, and the password will <b>always</b> be required when attempting to login as some user other than the logged-in user on authenticated notebook servers.
</font>

In [3]:
# An example of using the stored token to login to the Data Lab
tok2 = authClient.login('anonymous')
print ('New login token: %s' % tok2)
print ('Is new token valid?  %s' % authClient.isValidToken (tok2))

New login token: anonymous.0.0.anon_access
Is new token valid?  True


#### Additional Interface Methods

Users can check whether specific account information is correct:

In [4]:
print ('Is "demo00" a valid user?  %s' % authClient.isValidUser ('demo00'))
print ('Is "foobar" the current password?  %s' % authClient.isValidPassword ('demo00','foobar'))
print ('Do we have a valid login token?  %s' % authClient.isValidToken (token))

Is "demo00" a valid user?  True
Is "foobar" the current password?  False
Do we have a valid login token?  True


Users can also check on their login status using either the account name or the current token:

In [5]:
print ('Is user "dldemo" currently logged-in?  %s' % authClient.isUserLoggedIn ('demo00'))
print ('Is my token currently logged-in:  %s' % authClient.isTokenLoggedIn (token))
print ('Is user "Scooby" currently logged-in?  %s' % authClient.isUserLoggedIn ('Scooby'))

Is user "dldemo" currently logged-in?  True
Is my token currently logged-in:  True
Is user "Scooby" currently logged-in?  False


#### Resetting the User Account Password

An account password can be reset by presenting a valid token for the account, the account name and the new password.  If the password reset succeeds, a new identity token is returned to the task. A failed reset will return an error message so it is advisable to check the return value before overwriting the current token.

In [6]:
new_token = authClient.passwordReset (token, 'demo00', 'datalab')
if not authClient.isValidToken (new_token):
    print ('Error: password reset failed (%s)' % new_token)
else:
    print ('New Login token: %s' % new_token)
    print ('Old Login token: %s' % token)
    
print ('Is new token currently logged in: ' + authClient.isTokenLoggedIn (new_token))    

New Login token: demo00.1018.1018.$1$k3LeRBKt$ksi16AiRR6d8pXn0vpQ4d/
Old Login token: demo00.1018.1018.$1$Dyy.y/qT$z7u.OUM3y69TOXOV.zRL4.
Is new token currently logged in: True


We need to remember to reset the account password here so the next time we run the notebook we can login at the first cell.

In [7]:
token = authClient.passwordReset (new_token, 'demo00', 'balatad')
if not authClient.isValidToken (token):
    print ('Error: password reset failed (%s)' % token)
print ('Token:  %s' % token)
print ('Is the reset token currently logged in: ' + authClient.isTokenLoggedIn (token)) 
print ('Is the old reset token currently logged in: ' + authClient.isTokenLoggedIn (new_token))

# In order to run this cell multiple times, we'll reset the 'new_token' 
# so the next invocation of the reset works properly.
new_token = token

Token:  demo00.1018.1018.$1$Yy0EOfOo$XzAcDNHsKYXgV.5ppJq5p/
Is the reset token currently logged in: True
Is the old reset token currently logged in: False


#### Logging <i>Out</i> of the Data Lab

The `logout()` method is used to log the user out of the Data Lab and remove the identity token from the server.  Strictly speaking, it is not required that an application log out once it is complete, subsequent logins will return the same identity token the next time you log in so long as it is still valid (e.g. you haven't logged out elsewhere)

In [8]:
# Log out the current user and remove the identity token.
print ('Is token logged in: ' + authClient.isTokenLoggedIn (token))
res = authClient.logout (token)
print ('Log out result = "%s"'% res)
print ('Is token logged in following logout(): ' + authClient.isTokenLoggedIn (token))

Is token logged in: True
Log out result = "OK"
Is token logged in following logout(): False


### Example:  Anonymous Data Query

All public data services may be accessed anonymously, when queried in this way data will always be returned immediately to the client.  When saving results to a Data Lab resource such as virtual storage, however, a valid user login token is required.

In [9]:
anon_token = authClient.login ('anonymous')
query = "select * from usno.a2 limit 3"
response = queryClient.query (anon_token, adql=query, fmt='csv', output='foo.csv')
print (response)

id,raj2000_,dej2000_,actflag,mflag,bmag,rmag,epoch,raj2000,dej2000
1275-16995435,22:04:54.538,+42:13:16.11, , ,16.6,16,1952.653,331.227239,42.221142
1275-16995600,22:04:54.937,+42:13:23.96, , ,19.299999,19.1,1952.653,331.228903,42.223323000000001
1275-16998356,22:05:01.470,+42:12:18.74, , ,15.8,14.5,1952.653,331.256125,42.205205999999997



### Example:  Saving Results to Virtual Storage

Details of using the Virtual Storage system are described elsewhere, but in general the term *Virtual Storage* is used to refer to either a user's private database (their *MyDB*), or a remote file storage service that resides close to the data services.  *Virtual Storage* is used to allow workflows to be created that minimize the transfer of data.  

#### Saving to the remote file storage

Saving results to a file can be done by specifying the 'out' parameter in the form:

`vos://<filename>`

The `<filename>` must be unique for each query or else an error will be issued and the results will not be saved.  Therefore, good coding style would trap potential errors and handle it accordingly, e.g.

In [22]:
query = "select * from usno.b1 limit 20"
try:
    response = queryClient.query (token, adql=query, fmt='csv', 
                                  out='vos://mags.csv')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)           # print the response
    else:
        print ("OK")

OK


#### Saving to the user's remote database

Similarly, results may be saved to a user's MyDB database by specifying a table name of the form:

`mydb://<tablename>`

As with filenames, the `<tablename>` must be unique.

In [23]:
query = "select * from usno.b1 limit 1000"
try:
    response = queryClient.query (token, adql=query, fmt='csv', 
                                  out='mydb://usno_test')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)           # print the response
    else:
        print ("OK")

Error: relation "usno_test" already exists



***

### Auth Client API Summary

A summary of the complete *AuthClient* API is shown below:

```
LOGIN METHODS:
--------------
             token = login  (user, password=None, debug=False, verbose=False)
               res =logout  (token)

VALIDATION METHODS:
-------------------
            bval = isAlive  (svc_url=DEF_SERVICE_URL)
       bval = isValidToken  (token)
        bval = isValidUser  (user)
    bval = isValidPassword  (user, password)

ACCESS METHODS:
---------------
          bval = hasAccess  (user, resource)
     bval = isUserLoggedIn  (user)
    bval = isTokenLoggedIn  (token)

ACCOUNT ADMIN METHODS:
----------------------
 new_token = passwordReset  (token, username, password)

         url = set_service  (svc_url)
               get_service  ()

               set_profile  (profile)
profile_name = get_profile  ()
 prof_list = list_profiles  (token)

To instantiate a Client object:

    authClient = getClient  ()
```